In [ ]:
# if (!requireNamespace("BiocManager"))
#  install.packages("BiocManager")
# BiocManager::install()

In [ ]:
# BiocManager::install(c("DESeq2", "GenomicFeatures", "AnnotationDbi"))
# BiocManager::install(c("org.Dr.eg.db"))
# BiocManager::install(c("apeglm"))
# BiocManager::install(c("pheatmap"))
# BiocManager::install("ReportingTools")
# BiocManager::install("pcaExplorer")

In [2]:
library("DESeq2")
library("apeglm")
library("ggplot2")
library("org.Dr.eg.db")
library("ReportingTools")
library("pcaExplorer")
library("purrr")

In [3]:
setwd("/Users/angueyraaristjm/Documents/LiMolec/zfRNAseq/20181018/20181018_DESeq2/")
directory <- "/Users/angueyraaristjm/Documents/LiMolec/zfRNAseq/20181018/20181018_DESeq2/"
getwd()

In [5]:
countData <- as.matrix(read.csv("gCount.csv", row.names="gene_id"))
colData <- read.csv("PHENO_DATA.csv", sep="\t", row.names=1)
colData

photoreceptor tgLine  gtf_file   
UV01 UV            sws1GFP 04_zfUV.gtf
UV02 UV            sws1GFP 05_zfUV.gtf
UV03 UV            sws1GFP 06_zfUV.gtf
UV04 UV            sws1GFP 07_zfUV.gtf
S01  S             sws2GFP 08_zfS.gtf 
S02  S             sws2GFP 09_zfS.gtf 
S03  S             sws2GFP 10_zfS.gtf 
S04  S             sws2GFP 11_zfS.gtf

In [6]:
countData <- countData[, rownames(colData)]
all(rownames(colData) == colnames(countData))
dds <- DESeqDataSetFromMatrix(countData = countData, colData = colData, design = ~photoreceptor)
dds <- DESeq(dds)
res <- results(dds)
summary(res)
# sum(res$padj < 0.1, na.rm=TRUE)
# write.csv(res, file = "/data/angueyraaristjm/20181018_DESeq2/results.csv")

[1] TRUE

estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing



out of 13520 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)       : 211, 1.6%
LFC < 0 (down)     : 129, 0.95%
outliers [1]       : 3044, 23%
low counts [2]     : 2044, 15%
(mean count < 4)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results



In [7]:
resOrdered <- res[order(res$padj),]
# write.csv(resOrdered, file = "results_allversions/results.csv")

### Save results (but first include genenames in results)

In [8]:
# Run this only if things have changed:
# resdata <- merge(as.data.frame(res), as.data.frame(counts(dds,normalized =TRUE)), by = 'row.names', sort = FALSE)
# resdata <- resdata[order(resdata$padj),]
# names(resdata)[1] <- 'symbol'
# genenames <- mapIds(org.Dr.eg.db, keys=resdata[,c("symbol")], column=c("GENENAME"), keytype="SYMBOL")
# write.csv(genenames, file = "genenames.csv", col.names=c("symbol","genename"))
genenames <- read.csv("genenames.csv", sep=",")
colnames(genenames) <- c("symbol","genename")
head(genenames)

symbol           
1 abcc8b           
2 hnrnpul1         
3 msto1            
4 mib2             
5 si:ch1073-342h5.2
6 fkbp14           
  genename                                                     
1 ATP-binding cassette, sub-family C (CFTR/MRP), member 8b     
2 heterogeneous nuclear ribonucleoprotein U-like 1             
3 misato 1, mitochondrial distribution and morphology regulator
4 mindbomb E3 ubiquitin protein ligase 2                       
5 si:ch1073-342h5.2                                            
6 FK506 binding protein 14

In [9]:
# results + raw counts
resdata <- merge(as.data.frame(res), as.data.frame(counts(dds,normalized =FALSE)), by = 'row.names', sort = FALSE)
resdata <- resdata[order(resdata$padj),]
names(resdata)[1] <- 'symbol'

if (all(genenames$symbol == resdata$symbol)) {
    print("data frames DO match")
    resdata$genename = genenames$genename
    resdata <- resdata[c(1,ncol(resdata),2:ncol(resdata)-1)] #not sure why it's adding symbol again
    resdata <- resdata[c(1,2,4:ncol(resdata))]
    head(resdata)
    write.csv(resdata, file = "results_allversions/results_rawdata.csv")
} else {
    print("data frames do NOT match")
}

[1] "data frames DO match"


In [10]:
# results + normalized counts log2((counts/average sequencing depth across samples)+0.5)
resdata <- merge(as.data.frame(res), as.data.frame(counts(dds,normalized =TRUE)), by = 'row.names', sort = FALSE)
resdata <- resdata[order(resdata$padj),]
names(resdata)[1] <- 'symbol'

if (all(genenames$symbol == resdata$symbol)) {
    print("data frames DO match")
    resdata$genename = genenames$genename
    resdata <- resdata[c(1,ncol(resdata),2:ncol(resdata)-1)] #not sure why it's adding symbol again
    resdata <- resdata[c(1,2,4:ncol(resdata))]
    head(resdata)
    write.csv(resdata, file = "results_allversions/results_normdata.csv")
} else {
    print("data frames do NOT match")
}

[1] "data frames DO match"


In [11]:
head(resdata)

symbol           
1386 abcc8b           
1770 hnrnpul1         
2839 msto1            
4289 mib2             
5095 si:ch1073-342h5.2
5213 fkbp14           
     genename                                                      baseMean 
1386 ATP-binding cassette, sub-family C (CFTR/MRP), member 8b      142.31274
1770 heterogeneous nuclear ribonucleoprotein U-like 1              267.56757
2839 misato 1, mitochondrial distribution and morphology regulator 130.28707
4289 mindbomb E3 ubiquitin protein ligase 2                         90.28003
5095 si:ch1073-342h5.2                                              79.75580
5213 FK506 binding protein 14                                      142.12461
     log2FoldChange lfcSE    stat      pvalue       padj         UV01    
1386  24.44194      3.383648  7.223549 5.064821e-13 1.767334e-10 340.8746
1770  25.32257      3.383518  7.484095 7.204198e-14 1.767334e-10   0.0000
2839  24.31773      3.383673  7.186784 6.633501e-13 1.767334e-10   0.0000
4289 -24.29415      3.376163 -7.195787 6.210140e-13 1.767334e-10   0.0000
5095 -24.12568      3.346402 -7.209438 5.618340e-13 1.767334e-10   0.0000
5213 -24.91801      3.383614 -7.364320 1.780526e-13 1.767334e-10   0.0000
     UV02      UV03     UV04     S01 S02      S03      S04     
1386  797.6273   0.0000   0.0000 0     0.0000   0.0000   0.0000
1770 1457.7327   0.0000 682.8079 0     0.0000   0.0000   0.0000
2839  581.7179 460.5787   0.0000 0     0.0000   0.0000   0.0000
4289    0.0000   0.0000   0.0000 0   362.2895 359.9508   0.0000
5095    0.0000   0.0000   0.0000 0   395.4993   0.0000 242.5471
5213    0.0000   0.0000   0.0000 0     0.0000 412.9668 724.0301

In [12]:
# Log fold change shrinkage for visualization and ranking
# resultsNames(dds)
resLFC <- lfcShrink(dds, coef="photoreceptor_UV_vs_S", type="apeglm")
resLFC
# write.csv(resLFC, file = "results_LFCestimates.csv")
# REFERENCE: Zhu, A., Ibrahim, J.G., Love, M.I. (2018) Heavy-tailed prior distributions for sequence count data: removing the noise and preserving large differences. bioRxiv. 10.1101/303255

using 'apeglm' for LFC shrinkage. If used in published research, please cite:
    Zhu, A., Ibrahim, J.G., Love, M.I. (2018) Heavy-tailed prior distributions for
    sequence count data: removing the noise and preserving large differences.
    Bioinformatics. https://doi.org/10.1093/bioinformatics/bty895


log2 fold change (MAP): photoreceptor UV vs S 
Wald test p-value: photoreceptor UV vs S 
DataFrame with 28442 rows and 5 columns
                         baseMean     log2FoldChange            lfcSE
                        <numeric>          <numeric>        <numeric>
foxq2            1592.38581303657 -0.724379947935154 1.41423188013175
CU657925.1                      0                 NA               NA
neurl2                          0                 NA               NA
neurl4           22.7952108960837 -0.303278311490007  1.0888752633247
lamc2                           0                 NA               NA
...                           ...                ...              ...
si:ch73-314g15.3                0                 NA               NA
ptgdsa                          0                 NA               NA
abcb6b                          0                 NA               NA
abcb6a            5.1936653738911  0.299499622852028 1.08399975657478
ccdc87                         

In [13]:
# results + normalized counts log2((counts/average sequencing depth across samples)+0.5)
resdata <- merge(as.data.frame(resLFC), as.data.frame(counts(dds,normalized =TRUE)), by = 'row.names', sort = FALSE)
resdata <- resdata[order(resdata$padj),]
names(resdata)[1] <- 'symbol'

if (all(genenames$symbol == resdata$symbol)) {
    print("data frames DO match")
    resdata$genename = genenames$genename
    resdata <- resdata[c(1,ncol(resdata),2:ncol(resdata)-1)] #not sure why it's adding symbol again
    resdata <- resdata[c(1,2,4:ncol(resdata))]
    head(resdata)
    write.csv(resdata, file = "results_allversions/results_LFC_normdata.csv")
} else {
    print("data frames do NOT match")
}

[1] "data frames DO match"


In [14]:
head(resdata)

symbol           
1386 abcc8b           
1770 hnrnpul1         
2839 msto1            
4289 mib2             
5095 si:ch1073-342h5.2
5213 fkbp14           
     genename                                                      baseMean 
1386 ATP-binding cassette, sub-family C (CFTR/MRP), member 8b      142.31274
1770 heterogeneous nuclear ribonucleoprotein U-like 1              267.56757
2839 misato 1, mitochondrial distribution and morphology regulator 130.28707
4289 mindbomb E3 ubiquitin protein ligase 2                         90.28003
5095 si:ch1073-342h5.2                                              79.75580
5213 FK506 binding protein 14                                      142.12461
     log2FoldChange lfcSE    pvalue       padj         UV01     UV02     
1386  0.3181714     1.095263 5.064821e-13 1.767334e-10 340.8746  797.6273
1770  0.3205617     1.096496 7.204198e-14 1.767334e-10   0.0000 1457.7327
2839  0.3178590     1.095100 6.633501e-13 1.767334e-10   0.0000  581.7179
4289 -0.3058303     1.090592 6.210140e-13 1.767334e-10   0.0000    0.0000
5095 -0.3115681     1.093309 5.618340e-13 1.767334e-10   0.0000    0.0000
5213 -0.3023876     1.089036 1.780526e-13 1.767334e-10   0.0000    0.0000
     UV03     UV04     S01 S02      S03      S04     
1386   0.0000   0.0000 0     0.0000   0.0000   0.0000
1770   0.0000 682.8079 0     0.0000   0.0000   0.0000
2839 460.5787   0.0000 0     0.0000   0.0000   0.0000
4289   0.0000   0.0000 0   362.2895 359.9508   0.0000
5095   0.0000   0.0000 0   395.4993   0.0000 242.5471
5213   0.0000   0.0000 0     0.0000 412.9668 724.0301

In [15]:
#rounding for excel displaying
resrounded <-resdata
resrounded["baseMean"] <- purrr::modify_if(resrounded["baseMean"], ~is.numeric(.), ~round(., 2))
resrounded["log2FoldChange"] <- purrr::modify_if(resrounded["log2FoldChange"], ~is.numeric(.), ~round(., 4))
resrounded["lfcSE"] <- purrr::modify_if(resrounded["lfcSE"], ~is.numeric(.), ~round(., 4))
resrounded["UV01"] <- purrr::modify_if(resrounded["UV01"], ~is.numeric(.), ~round(., 0))
resrounded["UV02"] <- purrr::modify_if(resrounded["UV02"], ~is.numeric(.), ~round(., 0))
resrounded["UV03"] <- purrr::modify_if(resrounded["UV03"], ~is.numeric(.), ~round(., 0))
resrounded["UV04"] <- purrr::modify_if(resrounded["UV04"], ~is.numeric(.), ~round(., 0))
resrounded["S01"] <- purrr::modify_if(resrounded["S01"], ~is.numeric(.), ~round(., 0))
resrounded["S02"] <- purrr::modify_if(resrounded["S02"], ~is.numeric(.), ~round(., 0))
resrounded["S03"] <- purrr::modify_if(resrounded["S03"], ~is.numeric(.), ~round(., 0))
resrounded["S04"] <- purrr::modify_if(resrounded["S04"], ~is.numeric(.), ~round(., 0))
head(resrounded)
write.csv(resrounded, file = "results_allversions/results_LFC_norm_round.csv")



symbol           
1386 abcc8b           
1770 hnrnpul1         
2839 msto1            
4289 mib2             
5095 si:ch1073-342h5.2
5213 fkbp14           
     genename                                                      baseMean
1386 ATP-binding cassette, sub-family C (CFTR/MRP), member 8b      142.31  
1770 heterogeneous nuclear ribonucleoprotein U-like 1              267.57  
2839 misato 1, mitochondrial distribution and morphology regulator 130.29  
4289 mindbomb E3 ubiquitin protein ligase 2                         90.28  
5095 si:ch1073-342h5.2                                              79.76  
5213 FK506 binding protein 14                                      142.12  
     log2FoldChange lfcSE  pvalue       padj         UV01 UV02 UV03 UV04 S01
1386  0.3182        1.0953 5.064821e-13 1.767334e-10 341   798   0    0  0  
1770  0.3206        1.0965 7.204198e-14 1.767334e-10   0  1458   0  683  0  
2839  0.3179        1.0951 6.633501e-13 1.767334e-10   0   582 461    0  0  
4289 -0.3058        1.0906 6.210140e-13 1.767334e-10   0     0   0    0  0  
5095 -0.3116        1.0933 5.618340e-13 1.767334e-10   0     0   0    0  0  
5213 -0.3024        1.0890 1.780526e-13 1.767334e-10   0     0   0    0  0  
     S02 S03 S04
1386   0   0   0
1770   0   0   0
2839   0   0   0
4289 362 360   0
5095 395   0 243
5213   0 413 724

In [20]:
## save useful versions
# write.csv(resrounded, file = "results_userfriendly/zfUVvsS_sorted.csv")
# write.csv(resrounded[order(resrounded$symbol),], file = "results_userfriendly/zfUVvsS_alfabethical.csv")

# respvalue <- subset(resrounded, padj<0.1)
# write.csv(respvalue[order(respvalue$log2FoldChange),], file = "results_userfriendly/zfUVvsS_pvalue_FCsorted.csv")
respvalueS <- subset(respvalue, log2FoldChange<0)
write.csv(respvalueS[order(respvalueS$baseMean),], file = "results_userfriendly/zfUVvsS_pvalue_S.csv")
respvalueU <- subset(respvalue, log2FoldChange>0)
write.csv(respvalueU[order(respvalueU$baseMean),], file = "results_userfriendly/zfUVvsS_pvalue_UV.csv")

# HERE

In [18]:
dim(subset(resrounded, padj<0.1))

[1] 340  15

#### add gene names to data frames

In [ ]:
columns(org.Dr.eg.db)

In [ ]:
# genenames <- mapIds(org.Dr.eg.db, keys=resdata[,c("symbol")], column=c("GENENAME"), keytype="SYMBOL")
# write.csv(genenames, file = "genenames.csv", col.names=c("symbol","genename"))
genenames <- read.csv("genenames.csv", sep=",")
colnames(genenames) <- c("symbol","genename")
head(genenames)
all(genenames$symbol == resdata$symbol)

# head(resdata)
# entrezid <- mapIds(org.Dr.eg.db, keys=resdata[,c("Row.names")], column=c("ZFIN"), keytype="SYMBOL")
# head(entrezid)
# merge(res, annots, by.x=0, by.y="SYMBOL")


# test <- merge(resdata,genenames)

In [ ]:
tres <- head(resdata)
tgn <- head(genenames)
tres$genename = tgn$genename
# tres <- tres[, colnames(tres)[c(1,ncol(tres),2:ncol(tres)-1)]]
# tres <- tres[c(1,ncol(tres),2:ncol(tres)-1)]
tres <- tres[c(1,ncol(tres),2:ncol(tres)-1)] #not sure why it's adding symbol again
tres <- tres[c(1,2,4:ncol(tres))]
tres

In [ ]:
# MA plot
#log2 fold changes attributable to a given variable over the mean of normalized counts for all the samples in the DESeqDataSet. Points will be colored red if the adjusted p value is less than 0.1. Points which fall out of the window are plotted as open triangles pointing either up or down.
# plotMA(res, ylim=c(-20,20)) 
# to remove noise from log2 changes, it's better to visualize shrunken version
test <- plotMA(resLFC, ylim=c(-20,20))

#Plot is supposed to be interactive and genes could be recovered using this:
# idx <- identify(res$baseMean, res$log2FoldChange)
# rownames(res)[idx]

In [ ]:
# plot a single gene: counts (normalized by seq depth and +0.5 for log plotting)
test <- plotCounts(dds, gene="rho", intgroup="photoreceptor", col =c('blue','blue'), fg='white', col.lab ='white', col.main ='white', col.sub ='white', col.axis='white', bg='white')

In [ ]:
# more customizable plot of a single gene: counts (normalized by seq depth and +0.5 for log plotting)
data <- plotCounts(dds, gene="opn1sw1", intgroup=c("photoreceptor"), returnData=TRUE)
data
ggplot(data, aes(x=photoreceptor, y=count, color=photoreceptor)) +
  scale_y_log10() + 
  geom_point(position=position_jitter(width=.1,height=0))

In [ ]:
# mcols(resLFC)$description
head(df)

In [ ]:
# visualize as heatmap
n_genes = 10
library("pheatmap")
ntd <- normTransform(dds)
select <- order(rowMeans(counts(dds,normalized=TRUE)),
                decreasing=TRUE)[1:n_genes]
df <- as.data.frame(colData(dds)[c("photoreceptor")])
assay(ntd)[select,]
pheatmap(assay(ntd)[select,], cluster_rows=FALSE, show_rownames=FALSE,
         cluster_cols=FALSE, annotation_col=df)

In [ ]:
# sampleDists <- dist(t(assay(ntd)))
# sampleDists
# library("RColorBrewer")
# sampleDistMatrix <- as.matrix(sampleDists)
# rownames(sampleDistMatrix) <- paste(ntd$photoreceptor, sep="-")
# colnames(sampleDistMatrix) <- NULL
# colors <- colorRampPalette( rev(brewer.pal(9, "Blues")) )(255)
# pheatmap(sampleDistMatrix,
#          clustering_distance_rows=sampleDists,
#          clustering_distance_cols=sampleDists,
#          col=colors)
pcaData

In [ ]:
#built-in
# plotPCA(ntd, intgroup=c("photoreceptor"))

##ggplot
pcaData <- plotPCA(ntd, intgroup=c("photoreceptor"), returnData=TRUE)
percentVar <- round(100 * attr(pcaData, "percentVar"))
ggplot(pcaData, aes(PC1, PC2, color=photoreceptor)) +
  geom_point(size=3) +
  xlab(paste0("PC1: ",percentVar[1],"% variance")) +
  ylab(paste0("PC2: ",percentVar[2],"% variance")) + 
  coord_fixed()

In [ ]:
plotDispEsts(dds)
#probably means that data should be prefiltered. Maybe remove all lines with very low counts

In [ ]:
pcaExplorer(dds = dds)